In [ ]:
import pygame
from pygame.constants import K_0, K_1, K_2, K_3, K_4, K_LEFT, K_RIGHT, K_UP, K_DOWN
from math import floor
import time
from collections import deque
import random

#Initialisation
pygame.init()
screen = pygame.display.set_mode((800, 800))
horloge = pygame.time.Clock()

#Obstacles 
obstacles_lignes = [ [0, 4, 10, 16], [0, 14, 16], [0, 6, 16], [0, 9, 16], [0, 3, 15, 16], [0, 7, 16], [0, 1, 6, 9, 12, 16], [0, 6, 9, 16], [0, 6, 9, 16], [0, 4, 13, 16], [0, 6, 16], [0, 10, 16], [0, 8, 16], [0, 2, 15, 16], [0, 4, 10, 16], [0, 5, 12, 16] ]
obstacles_colonnes = [ [0, 5, 11, 16], [0, 6, 13, 16], [0, 4, 16], [0, 15, 16], [0, 10, 16], [0, 3, 16], [0, 6, 9, 10, 16], [0, 6, 9, 12, 16], [0, 6, 9, 16], [0, 3, 12, 16], [0, 14, 16], [0, 16], [0, 7, 16], [0, 2, 10, 16], [0, 4, 13, 16], [0, 2, 12, 16] ]

#Positions initiales en pixels
xR, yR = 25, 25  
xB, yB = 75, 25  
xJ, yJ = 175, 25  
xN, yN = 125, 25  


def jeu_to_coord(t):
    return (t[1] * 50 + 25, t[0] * 50 + 25)

# Obstacles robots
robR_obstacles = [(xB, yB), (xJ, yJ), (xN, yN)]
robB_obstacles = [(xR, yR), (xJ, yJ), (xN, yN)]
robJ_obstacles = [(xR, yR), (xB, yB), (xN, yN)]
robN_obstacles = [(xR, yR), (xB, yB), (xJ, yJ)]



#Positions précédentes
previous_xR, previous_yR = xR, yR
previous_xB, previous_yB = xB, yB
previous_xJ, previous_yJ = xJ, yJ
previous_xN, previous_yN = xN, yN

#Flags pour sélectionner le robot actif 
FlagR, FlagB, FlagJ, FlagN = True, False, False, False  # Rouge sélectionné par défaut
Flag_left, Flag_right, Flag_up, Flag_down = False, False, False, False


background_color = (255, 255, 255)


#Fonctions utilitaires
def cree_obstacle_ligne(a, b):
    pygame.draw.rect(screen, (0, 0, 0), (50 * a, 50 * b, 5, 50))


def cree_obstacle_col(a, b): 
    pygame.draw.rect(screen, (0, 0, 0), (50 * a, 50 * b, 50, 5))


def dichotomie(a, t):
    n = len(t)
    b, c = 0, n
    i = (b + c) // 2
    while t[i] > a or a >= t[i + 1]:
        if t[i] <= a:
            b = i
            i = (i + c) // 2
        else:
            c = i
            i = (b + i) // 2
    return i


#retourne les déplacements possible a partir d'une position (sans compter les robots)
def deplacements_grille(x, y):
    b = floor(x / 50)
    a = floor(y / 50)
    i = dichotomie(b, obstacles_lignes[a])
    ouest = (a, obstacles_lignes[a][i])
    est = (a, obstacles_lignes[a][i + 1] - 1)
    j = dichotomie(a, obstacles_colonnes[b])
    nord = (obstacles_colonnes[b][j], b)
    sud = (obstacles_colonnes[b][j + 1] - 1, b)
    return (ouest, est, nord, sud)


#modifie les déplacements précédents en fonction des robots présents 
def modif(t, a, b, c, d):
    ouest, est, nord, sud = t
    if a == c:
        if d < b:
            ouest = (a, max(ouest[1], d + 1))
        elif d > b:
            est = (a, min(est[1], d - 1))
    if b == d:
        if c < a:
            nord = (max(nord[0], c + 1), b)
        elif c > a:
            sud = (min(sud[0], c - 1), b)
    return (ouest, est, nord, sud)


#renvoie les 4 deplacements possibles d'un robot en coordonnées x,y
def deplacement_robot(x, y, q):
    a = floor(y / 50)
    b = floor(x / 50)
    depl = deplacements_grille(x, y)
    if q == []:
        return depl
    q_grille = [(floor(r[1] / 50), floor(r[0] / 50)) for r in q]
    for (c, d) in q_grille:
        depl = modif(depl, a, b, c, d)
    return depl


#prend un sommet défini comme : position du robot principale "robot", liste des autres robots 
def sommets_accessibles(sommet):
    robot_pp = sommet["robot"]
    autres_robots = sommet["autres_robots"]
    resultats = []
    robot_pp_pixels = jeu_to_coord(robot_pp)
    autres_pixels = [jeu_to_coord(r) for r in autres_robots]
    deplacements_pp = deplacement_robot(robot_pp_pixels[0], robot_pp_pixels[1], autres_pixels)
    for depl in deplacements_pp:
        resultats.append({"robot": depl, "autres_robots": autres_robots.copy()})
    for i in range(len(autres_robots)):
        robot = autres_robots[i]
        robot_pixels = jeu_to_coord(robot)
        autres = autres_robots[:i] + autres_robots[i + 1:]
        autres_pixels = [jeu_to_coord(r) for r in autres] + [jeu_to_coord(robot_pp)]
        deplacements = deplacement_robot(robot_pixels[0], robot_pixels[1], autres_pixels)
        for depl in deplacements:
            new_autres = autres_robots.copy()
            new_autres[i] = depl
            new_autres.sort()
            resultats.append({"robot": robot_pp, "autres_robots": new_autres})
    return resultats


#calcul du chemin optimal en stockant les sommets dans des tables de hachages.
def bfs(robot_pp, autres_robots, objectif):
    queue = deque([{"robot": robot_pp, "autres_robots": autres_robots, "chemin": []}])
    visites = set()
    while queue:
        sommet = queue.popleft()
        robot_position = sommet["robot"]
        autres_positions = sommet["autres_robots"]
        chemin = sommet["chemin"]
        if robot_position == objectif:
            return chemin
        cle_sommet = (robot_position, tuple(autres_positions))
        if cle_sommet in visites: #ici en O(1) !
            continue
        visites.add(cle_sommet)
        accessibles = sommets_accessibles({"robot": robot_position, "autres_robots": autres_positions})
        for next_sommet in accessibles:
            move = None
            if next_sommet["robot"] != robot_position:
                move = ("robot_principal", next_sommet["robot"])
            else:
                for i, (before, after) in enumerate(zip(autres_positions, next_sommet["autres_robots"])):
                    if before != after:
                        move = (f"robot_{i+1}", after)
                        break
            queue.append({
                "robot": next_sommet["robot"],
                "autres_robots": next_sommet["autres_robots"],
                "chemin": chemin + [move]  # move est toujours défini maintenant
            })
    return None


# exemple de test



sommet_initial = {"robot": (xR//50, yR//50), "autres_robots": [(xJ//50, yJ//50), (xN//50, xN//50), (xB//50, xB//50)]}  # Rouge à (4, 4)  # 
objectif = (15, 3)  # Objectif exemple pour le robot rouge



# Exécution du BFS pour le robot rouge
start_time = time.time()

resultat = bfs(sommet_initial["robot"], sommet_initial["autres_robots"], objectif)

end_time = time.time()
elapsed_time = end_time - start_time

if resultat is None:
    print("t'es nul")
else:
    nb_etape = 0
    print("Chemin trouvé :")
    for etape in resultat:
        identite, position = etape
        print(f"{identite} → {position}")


print(f"Temps total d'exécution : {elapsed_time:.4f} secondes")



# Boucle principale
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    keys = pygame.key.get_pressed()
    if keys[K_0]:
        running = False

    # Mise à jour des flags selon les touches pressées
    if keys[K_1]:
        FlagR, FlagB, FlagJ, FlagN = True, False, False, False
    elif keys[K_2]:
        FlagR, FlagB, FlagJ, FlagN = False, True, False, False
    elif keys[K_3]:
        FlagR, FlagB, FlagJ, FlagN = False, False, True, False
    elif keys[K_4]:
        FlagR, FlagB, FlagJ, FlagN = False, False, False, True

    # Dessin de l'arrière-plan
    screen.fill(background_color)
    for i in range(0, 800, 50):
        for j in range(0, 800, 50):
            pygame.draw.rect(screen, (210, 210, 210), (i, j, 50, 50), 10)
            pygame.draw.rect(screen, (51, 50, 55), (i, j, 50, 50), 1)
            pygame.draw.rect(screen, (191, 191, 191), (10 + i, 10 + j, 30, 30), 2)
    pygame.draw.rect(screen, (197, 183, 174), (300, 300, 150, 150))

    # Dessin des obstacles fixes
    for i in range(len(obstacles_lignes)):
        for j in range(len(obstacles_lignes[i])):
            cree_obstacle_ligne(obstacles_lignes[i][j], i)
    for i in range(len(obstacles_colonnes)):
        for j in range(len(obstacles_colonnes[i])):
            cree_obstacle_col(i, obstacles_colonnes[i][j])

    mouvement_x, mouvement_y = 5, 5

    # Mouvements pour le robot rouge
    if FlagR:
        if previous_xR == xR and previous_yR == yR:
            objectifR = deplacement_robot(xR, yR, robR_obstacles)
            objectifR_pixels = [jeu_to_coord(pos) for pos in objectifR]
        for i in range(4):
            if keys[K_RIGHT] or Flag_right:
                if xR < objectifR_pixels[1][0]:  # Est
                    xR += mouvement_x
                    Flag_right = True
                else:
                    Flag_right = False
                    previous_xR, previous_yR = xR, yR
            if keys[K_LEFT] or Flag_left:
                if xR > objectifR_pixels[0][0]:  # Ouest
                    xR -= mouvement_x
                    Flag_left = True
                else:
                    Flag_left = False
                    previous_xR, previous_yR = xR, yR
            if keys[K_DOWN] or Flag_down:
                if yR < objectifR_pixels[3][1]:  # Sud
                    yR += mouvement_y
                    Flag_down = True
                else:
                    Flag_down = False
                    previous_xR, previous_yR = xR, yR
            if keys[K_UP] or Flag_up:
                if yR > objectifR_pixels[2][1]:  # Nord
                    yR -= mouvement_y
                    Flag_up = True
                else:
                    Flag_up = False
                    previous_xR, previous_yR = xR, yR

    # Mouvements pour le robot bleu
    elif FlagB:
        if previous_xB == xB and previous_yB == yB:
            objectifB = deplacement_robot(xB, yB, robB_obstacles)
            objectifB_pixels = [jeu_to_coord(pos) for pos in objectifB]
        for i in range(4):
            if keys[K_RIGHT] or Flag_right:
                if xB < objectifB_pixels[1][0]:  # Est
                    xB += mouvement_x
                    Flag_right = True
                else:
                    Flag_right = False
                    previous_xB, previous_yB = xB, yB
            if keys[K_LEFT] or Flag_left:
                if xB > objectifB_pixels[0][0]:  # Ouest
                    xB -= mouvement_x
                    Flag_left = True
                else:
                    Flag_left = False
                    previous_xB, previous_yB = xB, yB
            if keys[K_DOWN] or Flag_down:
                if yB < objectifB_pixels[3][1]:  # Sud
                    yB += mouvement_y
                    Flag_down = True
                else:
                    Flag_down = False
                    previous_xB, previous_yB = xB, yB
            if keys[K_UP] or Flag_up:
                if yB > objectifB_pixels[2][1]:  # Nord
                    yB -= mouvement_y
                    Flag_up = True
                else:
                    Flag_up = False
                    previous_xB, previous_yB = xB, yB

    # Mouvements pour le robot jaune
    elif FlagJ:
        if previous_xJ == xJ and previous_yJ == yJ:
            objectifJ = deplacement_robot(xJ, yJ, robJ_obstacles)
            objectifJ_pixels = [jeu_to_coord(pos) for pos in objectifJ]
        for i in range(4):
            if keys[K_RIGHT] or Flag_right:
                if xJ < objectifJ_pixels[1][0]:  # Est
                    xJ += mouvement_x
                    Flag_right = True
                else:
                    Flag_right = False
                    previous_xJ, previous_yJ = xJ, yJ
            if keys[K_LEFT] or Flag_left:
                if xJ > objectifJ_pixels[0][0]:  # Ouest
                    xJ -= mouvement_x
                    Flag_left = True
                else:
                    Flag_left = False
                    previous_xJ, previous_yJ = xJ, yJ
            if keys[K_DOWN] or Flag_down:
                if yJ < objectifJ_pixels[3][1]:  # Sud
                    yJ += mouvement_y
                    Flag_down = True
                else:
                    Flag_down = False
                    previous_xJ, previous_yJ = xJ, yJ
            if keys[K_UP] or Flag_up:
                if yJ > objectifJ_pixels[2][1]:  # Nord
                    yJ -= mouvement_y
                    Flag_up = True
                else:
                    Flag_up = False
                    previous_xJ, previous_yJ = xJ, yJ

    # Mouvements pour le robot noir
    elif FlagN:
        if previous_xN == xN and previous_yN == yN:
            objectifN = deplacement_robot(xN, yN, robN_obstacles)
            objectifN_pixels = [jeu_to_coord(pos) for pos in objectifN]
        for i in range(4):
            if keys[K_RIGHT] or Flag_right:
                if xN < objectifN_pixels[1][0]:  # Est
                    xN += mouvement_x
                    Flag_right = True
                else:
                    Flag_right = False
                    previous_xN, previous_yN = xN, yN
            if keys[K_LEFT] or Flag_left:
                if xN > objectifN_pixels[0][0]:  # Ouest
                    xN -= mouvement_x
                    Flag_left = True
                else:
                    Flag_left = False
                    previous_xN, previous_yN = xN, yN
            if keys[K_DOWN] or Flag_down:
                if yN < objectifN_pixels[3][1]:  # Sud
                    yN += mouvement_y
                    Flag_down = True
                else:
                    Flag_down = False
                    previous_xN, previous_yN = xN, yN
            if keys[K_UP] or Flag_up:
                if yN > objectifN_pixels[2][1]:  # Nord
                    yN -= mouvement_y
                    Flag_up = True
                else:
                    Flag_up = False
                    previous_xN, previous_yN = xN, yN

    # Mise à jour des obstacles dynamiques
    robR_obstacles = [(xB, yB), (xJ, yJ), (xN, yN)]
    robB_obstacles = [(xR, yR), (xJ, yJ), (xN, yN)]
    robJ_obstacles = [(xR, yR), (xB, yB), (xN, yN)]
    robN_obstacles = [(xR, yR), (xB, yB), (xJ, yJ)]

    # Dessin des robots
    pygame.draw.circle(screen, (255, 0, 0), (int(xR), int(yR)), 20)    # Rouge
    pygame.draw.circle(screen, (0, 0, 139), (int(xB), int(yB)), 20)    # Bleu
    pygame.draw.circle(screen, (253, 238, 0), (int(xJ), int(yJ)), 20)  # Jaune
    pygame.draw.circle(screen, (0, 0, 0), (int(xN), int(yN)), 20)      # Noir

    pygame.draw.rect(screen, (255,0,0),(7 + objectif[1]*50,7+ objectif[0]*50, 40, 40))


        # Initialisation des variables de mouvement automatique
    auto_mode = True  # Met à True pour démarrer automatiquement
    etape_index = 0
    vitesse_anim = 5  # pixels par frame

    # À insérer juste avant pygame.display.flip() dans ta boucle while :
    if auto_mode and resultat is not None and etape_index < len(resultat):
        identite, cible = resultat[etape_index]
        cible_x, cible_y = jeu_to_coord(cible)

        if identite == "robot_principal":
            dx = cible_x - xR
            dy = cible_y - yR
            if abs(dx) > 0:
                xR += vitesse_anim if dx > 0 else -vitesse_anim
            if abs(dy) > 0:
                yR += vitesse_anim if dy > 0 else -vitesse_anim
            if abs(dx) <= vitesse_anim and abs(dy) <= vitesse_anim:
                xR, yR = cible_x, cible_y
                etape_index += 1
        # Tu peux aussi ajouter ici le déplacement automatique des autres robots si besoin :
        # elif identite == "robot_1": etc.

        # Mettre à jour les obstacles dynamiques après mouvement
        robR_obstacles = [(xB, yB), (xJ, yJ), (xN, yN)]


    pygame.display.flip()
    pygame.time.delay(20)

pygame.quit()

ModuleNotFoundError: No module named 'bfs'

1
256
